In [17]:
import pandas as pd
import numpy as np
import re
import os

df = pd.read_json('../Data/Serie_A_2014_games.json')

df_squad = pd.read_json('../Data/Serie_A_2014_squads.json').T

df_transposto = df.transpose()

df_transposto.head(3)

,Home,Away,Result,Players,Goals,Changes,Yellow cards,Red cards
1,Flamengo / RJ,Goiás / GO,0 X 0,[[1Felipe Luiz Felipe Ventura ... T(g)P141698...,[],[45:00 INTFlamengo/RJ 17 - Gabriel Santana Pin...,"[24:00 1T40Mauricio Azevedo Alves Flamengo/RJ,...",[]
2,Fluminense / RJ,Figueirense / SC,3 X 0,"[[12Diego Diego Cavalieri T(g)P137990, Flumine...",[31:00 1T23NRRafael Augusto Sobis Fluminense/R...,[45:00 INTFigueirense/SC 15 - Paulo Roberto da...,[44:00 1T4Thiago Heleno Henrique Ferreira Figu...,[]
3,São Paulo / SP,Botafogo / RJ,3 X 0,"[[1Rogerio Rogerio Ceni T(g)P107639, São Paulo...",[12:00 1T4NRAntonio Carlos dos Santos Aguiar S...,[45:00 INTBotafogo/RJ 23 - Mario Ariel Bolatti...,[30:00 1T6Alvaro Daniel Pereira Barragan São P...,[]


In [18]:
df_squad.head(3)

,0,1,2,3,4,5,6,7,8
1,"{'Home': {'Squad': ['141698', '137644', '14474...","{'Home': {'Squad': ['141698', '137644', '14474...","{'Home': {'Squad': ['141698', '137644', '14474...","{'Home': {'Squad': ['141698', '137644', '14474...","{'Home': {'Squad': ['141698', '137644', '14474...","{'Home': {'Squad': ['141698', '137644', '14474...",NaN,NaN,NaN
2,"{'Home': {'Squad': ['137990', '172996', '16795...","{'Home': {'Squad': ['137990', '172996', '16795...","{'Home': {'Squad': ['137990', '172996', '16795...","{'Home': {'Squad': ['137990', '172996', '16795...","{'Home': {'Squad': ['137990', '172996', '16795...","{'Home': {'Squad': ['137990', '172996', '16795...",NaN,NaN,NaN
3,"{'Home': {'Squad': ['107639', '161940', '45966...","{'Home': {'Squad': ['107639', '161940', '45966...","{'Home': {'Squad': ['107639', '161940', '45966...","{'Home': {'Squad': ['107639', '161940', '31664...",NaN,NaN,NaN,NaN,NaN


Teste inicial utilizando um jogo do brasileirão Série A 2014. Pegamos o primeiro jogo do Fluminense x Figueirense para criarmos a formatação das colunas e extração de informações da súmula.

In [19]:
players_teams = df_transposto.iloc[1, 3]
team_changes = df_transposto.iloc[1, 5]

start_players = df_squad.iloc[1,:]

# Criando DataFrame dos jogadores
new_df_players = pd.DataFrame(players_teams, columns=['player', 'team'])

def extract_id(player_string):
    # Ajuste para garantir que captura corretamente todas as variantes de ID
    match = re.search(r'T\(g\)?P(\d+)|RP(\d+)|TP(\d+)', player_string, re.IGNORECASE)
    if match:
        return next((m for m in match.groups() if m), None)
    return None

# Limpeza dos nomes dos jogadores
def clean_player_name(player):
    clean_name = re.sub(r'\s+T.*|\s+TP.*|\s+RP.*|\s+T(g).*', '', player)  # Remove texto após 'T' ou 'TP'
    return clean_name

new_df_players['player_id'] = new_df_players['player'].apply(extract_id)
new_df_players['player_name'] = new_df_players['player'].apply(clean_player_name)


def parse_team_changes(changes):
    pattern = re.compile(r'(\d{2}:\d{2}) (INT|\d+T)([\w\s]+/\w+) (\d+) - [^\d]+ (\d+) - [^\d]+')
    parsed_data = []
    for change in changes:
        match = pattern.search(change)
        if match:
            time, half, team, player_out_number, player_in_number = match.groups()
            team = team.strip()  # Limpeza para remover espaços extras
            parsed_data.append((time, half, team, player_out_number, player_in_number))
    return parsed_data


parsed_changes = parse_team_changes(team_changes)
print(parsed_changes[0])


new_df_players['Minutes Played'] = 90
new_df_players['Minute Entered'] = 0  # Todos os jogadores começam com entrada no minuto 0
new_df_players['Minute Exited'] = 90  # Todos os jogadores começam com saída no minuto 90

for time, half, team, player_out_number, player_in_number in parsed_changes:
    minute = int(time.split(':')[0])
    team = team.replace('/', ' / ')  # Formatando o nome do time corretamente
    
    if '2T' in half:
        minute_entered = 45 + minute
        minute_exited = 45 + minute
    else:
        minute_entered = minute
        minute_exited = minute

    # Atualizar o jogador que entrou
    mask_in = (new_df_players['player_name'].apply(lambda x: re.match(r'^' + player_in_number + r'\D', x) is not None) & 
               (new_df_players['team'] == team))
    new_df_players.loc[mask_in, 'Minute Entered'] = minute_entered
    new_df_players.loc[mask_in, 'Minute Exited'] = 90  # O jogador que entra fica até o final do jogo

    # Atualizar o jogador que saiu
    mask_out = (new_df_players['player_name'].apply(lambda x: re.match(r'^' + player_out_number + r'\D', x) is not None) & 
                (new_df_players['team'] == team))
    new_df_players.loc[mask_out, 'Minute Exited'] = minute_exited

# Recalcular os minutos jogados com base nos minutos de entrada e saída
new_df_players['Minutes Played'] = new_df_players['Minute Exited'] - new_df_players['Minute Entered']

#setando Home ou Away
home_team = df_transposto.iloc[1, 0]
new_df_players
new_df_players['status'] = np.where(new_df_players['team'] == home_team, 'Home', 'Away')

new_df_players = new_df_players.drop(columns='player')
# Visualizando o novo DataFrame
new_df_players

('45:00', 'INT', 'Figueirense/SC', '15', '20')


,team,player_id,player_name,Minutes Played,Minute Entered,Minute Exited,status
0,Fluminense / RJ,137990,12Diego Diego Cavalieri,90,0,90,Home
1,Fluminense / RJ,172996,2Bruno Bruno Vieira do Nasc ...,90,0,90,Home
2,Fluminense / RJ,167958,3Welington ... Welington Pereira Ro ...,90,0,90,Home
3,Fluminense / RJ,173760,6Carlinhos Carlos Andrade Souza,90,0,90,Home
4,Fluminense / RJ,173180,7Jean Jean Raphael Vanderl ...,90,0,90,Home
5,Fluminense / RJ,147224,8Rodrigo Ol ... Rodrigo Oliveira de ...,8,82,90,Home
6,Fluminense / RJ,148994,9Fred Frederico Chaves Guedes,90,0,90,Home
7,Fluminense / RJ,154127,10Wagner Wagner Ferreira dos ...,23,67,90,Home
8,Fluminense / RJ,184395,11Conca Dario Leonardo Conca,90,0,90,Home
9,Fluminense / RJ,317763,14Elivelton Elivelton Viana dos ...,90,0,90,Home


In [20]:

def parse_goals(goals, home_team):
    pattern = re.compile(r'(\d+):00 (\d+T)([\d\w]+)([A-Za-z ]+) ([\w\s/]+)')
    parsed_goals = []
    for goal in goals:
        match = pattern.search(goal)
        if match:
            minute = int(match.group(1))
            half = match.group(2)
            scorer_info = match.group(3)
            player_name = match.group(4).strip()
            team = match.group(5).strip()
            team = team.replace('/', ' / ')
            
            # Convertendo para minuto total do jogo
            if '2T' in half and minute != 45:  # Ajusta para o tempo total no caso do segundo tempo
                minute += 45
            
            # Verificar se é um gol contra (CT)
            if 'CT' in scorer_info:
                if team == home_team:
                    team_status = 'Away'
                else:
                    team_status = 'Home'
            else:
                if team == home_team:
                    team_status = 'Home'
                else:
                    team_status = 'Away'
            
            parsed_goals.append((minute, team_status))
    return parsed_goals

home_team = df_transposto.iloc[1, 0]
parsed_goals = parse_goals(df_transposto.iloc[1,4], home_team)

parsed_goals

[(31, 'Home'), (44, 'Home'), (59, 'Home')]

In [21]:

new_df_players['Goals For'] = 0

new_df_players['Goals Against'] = 0

for minute, team in parsed_goals:
    # Jogadores em campo no momento do gol
    mask = (new_df_players['Minute Entered'] <= minute) & (new_df_players['Minute Exited'] >= minute)
    # Incrementar gols a favor
    mask_for = mask & (new_df_players['status'] == team)
    new_df_players.loc[mask_for, 'Goals For'] += 1
    
    # Incrementar gols contra
    mask_against = mask & (new_df_players['status'] != team)
    new_df_players.loc[mask_against, 'Goals Against'] += 1

# Exibindo o DataFrame final para verificar os resultados
new_df_players

,team,player_id,player_name,Minutes Played,Minute Entered,Minute Exited,status,Goals For,Goals Against
0,Fluminense / RJ,137990,12Diego Diego Cavalieri,90,0,90,Home,3,0
1,Fluminense / RJ,172996,2Bruno Bruno Vieira do Nasc ...,90,0,90,Home,3,0
2,Fluminense / RJ,167958,3Welington ... Welington Pereira Ro ...,90,0,90,Home,3,0
3,Fluminense / RJ,173760,6Carlinhos Carlos Andrade Souza,90,0,90,Home,3,0
4,Fluminense / RJ,173180,7Jean Jean Raphael Vanderl ...,90,0,90,Home,3,0
5,Fluminense / RJ,147224,8Rodrigo Ol ... Rodrigo Oliveira de ...,8,82,90,Home,0,0
6,Fluminense / RJ,148994,9Fred Frederico Chaves Guedes,90,0,90,Home,3,0
7,Fluminense / RJ,154127,10Wagner Wagner Ferreira dos ...,23,67,90,Home,0,0
8,Fluminense / RJ,184395,11Conca Dario Leonardo Conca,90,0,90,Home,3,0
9,Fluminense / RJ,317763,14Elivelton Elivelton Viana dos ...,90,0,90,Home,3,0


In [23]:

def collect_unique_ids(data_folder, file_names, years):
    unique_ids = set()

    def process_dataframe(df):
        for column in df.columns:
            for cell in df[column]:
                if isinstance(cell, dict):  
                    for key in ['Home', 'Away']:
                        team_info = cell.get(key, {})
                        # Adicionar os IDs do 'Squad'
                        squad_ids = team_info.get('Squad', [])
                        unique_ids.update(squad_ids)


    for file_name in file_names:
        for year in years:
            file_path = os.path.join(data_folder, f'{file_name}_{year}_squads.json')
            if os.path.exists(file_path):
                df = pd.read_json(file_path)
                df_ = df.iloc[:,1]
                df = pd.DataFrame(df_)
                process_dataframe(df)
            else:
                print(f'Arquivo não encontrado: {file_path}')

    return list(unique_ids)

data_folder = '../Data/'
file_names = ['Serie_A']
years = range(2014, 2015)

unique_player_ids = collect_unique_ids(data_folder, file_names, years)

print(f'Total de IDs únicos coletados: {len(unique_player_ids)}')

Total de IDs únicos coletados: 28


In [25]:

df_players_final = new_df_players[new_df_players['player_id'].isin(unique_player_ids)]
df_players_final

,team,player_id,player_name,Minutes Played,Minute Entered,Minute Exited,status,Goals For,Goals Against
0,Fluminense / RJ,137990,12Diego Diego Cavalieri,90,0,90,Home,3,0
1,Fluminense / RJ,172996,2Bruno Bruno Vieira do Nasc ...,90,0,90,Home,3,0
2,Fluminense / RJ,167958,3Welington ... Welington Pereira Ro ...,90,0,90,Home,3,0
3,Fluminense / RJ,173760,6Carlinhos Carlos Andrade Souza,90,0,90,Home,3,0
4,Fluminense / RJ,173180,7Jean Jean Raphael Vanderl ...,90,0,90,Home,3,0
5,Fluminense / RJ,147224,8Rodrigo Ol ... Rodrigo Oliveira de ...,8,82,90,Home,0,0
6,Fluminense / RJ,148994,9Fred Frederico Chaves Guedes,90,0,90,Home,3,0
7,Fluminense / RJ,154127,10Wagner Wagner Ferreira dos ...,23,67,90,Home,0,0
8,Fluminense / RJ,184395,11Conca Dario Leonardo Conca,90,0,90,Home,3,0
9,Fluminense / RJ,317763,14Elivelton Elivelton Viana dos ...,90,0,90,Home,3,0
